In [1]:
import os
import os.path as op
import shutil
import zipfile
import requests
from glob import glob

from nimare.utils import get_resource_path

In [2]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [3]:
coll = '1425'
url = 'https://neurovault.org/collections/{0}/download'.format(coll)
out_dir = op.join(get_resource_path(),
                  'data/neurovault-data/collection-{0}'.format(coll))

In [4]:
os.makedirs(out_dir, exist_ok=True)

# Download
fname = download_file(url)

# Unzip
with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall(out_dir)

collection_folders = [f for f in glob(op.join(out_dir, '*'))
                      if '.nidm' not in f]
collection_folders = [f for f in collection_folders if op.isdir(f)]
if len(collection_folders) > 1:
    raise Exception('More than one folder found: '
                    '{0}'.format(', '.join(collection_folders)))
else:
    folder = collection_folders[0]
zip_files = glob(op.join(folder, '*.zip'))
for zf in zip_files:
    fn = op.splitext(op.basename(zf))[0]
    with zipfile.ZipFile(zf, 'r') as zip_ref:
        zip_ref.extractall(op.join(out_dir, fn))
        
os.remove(fname)
shutil.rmtree(folder)